In [3]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [5]:
import os
if not os.path.isdir('saved_models_{}'.format('cogan')):
    os.mkdir('saved_models_{}'.format('cogan'))
f = open('saved_models_{}/log_collapse1.txt'.format('cogan'), mode='w')
import cv2
import torch.utils.data as Data

import scipy

from keras.datasets import cifar10,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt

import sys
import os

import numpy as np

class COGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/02/18/coupled_gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.d1, self.d2 = self.build_discriminators()
        self.d1.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d2.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.g1, self.g2 = self.build_generators()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img1 = self.g1(z)
        img2 = self.g2(z)

        # For the combined model we will only train the generators
        self.d1.trainable = False
        self.d2.trainable = False

        # The valid takes generated images as input and determines validity
        valid1 = self.d1(img1)
        valid2 = self.d2(img2)

        # The combined model  (stacked generators and discriminators)
        # Trains generators to fool discriminators
        self.combined = Model(z, [valid1, valid2])
        self.combined.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
                                    optimizer=optimizer)

    def build_generators(self):
        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img1 = model(noise)
        img2 = model(noise)



        return Model(noise, img1), Model(noise, img2)

    def build_discriminators(self):
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img1 = Input(shape=self.img_shape)
        img2 = Input(shape=self.img_shape)
        validity1 = model(img1)
        validity2 = model(img2)

        return Model(img1, validity1), Model(img2, validity2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        X_test = np.expand_dims(X_test, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Images in domain A and B (rotated)
        X1 = X_train[:int(X_train.shape[0]/2)]
        X2 = X_train[int(X_train.shape[0]/2):]
        X2 = scipy.ndimage.interpolation.rotate(X2, 90, axes=(1, 2))

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X1.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                # imgs = X_train[index * batch_size:(index + 1) * batch_size]

                imgs1 = X1[index * batch_size:(index + 1) * batch_size]
                imgs2 = X2[index * batch_size:(index + 1) * batch_size]
                # print(len(imgs1))

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a batch of new images
                gen_imgs1 = self.g1.predict(noise)
                gen_imgs2 = self.g2.predict(noise)

                # Train the discriminators
                d1_loss_real = self.d1.train_on_batch(imgs1, valid)
                d2_loss_real = self.d2.train_on_batch(imgs2, valid)
                d1_loss_fake = self.d1.train_on_batch(gen_imgs1, fake)
                d2_loss_fake = self.d2.train_on_batch(gen_imgs2, fake)
                d1_loss = 0.5 * np.add(d1_loss_real, d1_loss_fake)
                d2_loss = 0.5 * np.add(d2_loss_real, d2_loss_fake)

                # ------------------
                #  Train Generators
                # ------------------

                g_loss = self.combined.train_on_batch(noise, [valid, valid])

                # Plot the progress
                print("epoch:%d step:%d [D1 loss: %f, acc.: %.2f%%] [D2 loss: %f, acc.: %.2f%%] [G loss: %f]" \
                      % (epoch, global_step, d1_loss[0], 100 * d1_loss[1], d2_loss[0], 100 * d2_loss[1], g_loss[0]))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sampleSize):
        r, c =  sampleSize // 10,10
        noise = np.random.normal(0, 1, (r * int(c / 2), 100))
        gen_imgs1 = self.g1.predict(noise)
        gen_imgs2 = self.g2.predict(noise)
        gen_imgs = np.concatenate([gen_imgs1, gen_imgs2])
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sampleSize])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sampleSize], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
        

if __name__ == '__main__':
    gan = COGAN()
    gan.train(epochs=30, batch_size=64, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 64)          0         
__________

epoch:0 step:37 [D1 loss: 1.100121, acc.: 44.53%] [D2 loss: 0.769987, acc.: 57.03%] [G loss: 3.325433]
epoch:0 step:38 [D1 loss: 0.712384, acc.: 67.19%] [D2 loss: 0.582323, acc.: 73.44%] [G loss: 2.703358]
epoch:0 step:39 [D1 loss: 0.866662, acc.: 54.69%] [D2 loss: 0.519812, acc.: 73.44%] [G loss: 3.420436]
epoch:0 step:40 [D1 loss: 1.175160, acc.: 46.09%] [D2 loss: 0.693788, acc.: 68.75%] [G loss: 4.627790]
epoch:0 step:41 [D1 loss: 1.159957, acc.: 36.72%] [D2 loss: 0.825119, acc.: 51.56%] [G loss: 2.974707]
epoch:0 step:42 [D1 loss: 0.890749, acc.: 50.78%] [D2 loss: 0.490158, acc.: 77.34%] [G loss: 3.238165]
epoch:0 step:43 [D1 loss: 0.901864, acc.: 49.22%] [D2 loss: 0.525955, acc.: 74.22%] [G loss: 2.074206]
epoch:0 step:44 [D1 loss: 1.052943, acc.: 47.66%] [D2 loss: 0.366334, acc.: 84.38%] [G loss: 3.326713]
epoch:0 step:45 [D1 loss: 0.943487, acc.: 47.66%] [D2 loss: 0.546883, acc.: 74.22%] [G loss: 4.203671]
epoch:0 step:46 [D1 loss: 1.034218, acc.: 40.62%] [D2 loss: 0.566031, acc

epoch:0 step:118 [D1 loss: 0.922248, acc.: 44.53%] [D2 loss: 0.642218, acc.: 64.06%] [G loss: 2.730396]
epoch:0 step:119 [D1 loss: 0.907926, acc.: 42.97%] [D2 loss: 0.687426, acc.: 58.59%] [G loss: 3.115896]
epoch:0 step:120 [D1 loss: 0.922684, acc.: 39.06%] [D2 loss: 0.550066, acc.: 73.44%] [G loss: 3.236139]
epoch:0 step:121 [D1 loss: 0.889279, acc.: 39.06%] [D2 loss: 0.613774, acc.: 64.06%] [G loss: 2.542899]
epoch:0 step:122 [D1 loss: 0.704264, acc.: 61.72%] [D2 loss: 0.516331, acc.: 74.22%] [G loss: 2.948194]
epoch:0 step:123 [D1 loss: 0.819987, acc.: 47.66%] [D2 loss: 0.525662, acc.: 72.66%] [G loss: 3.094581]
epoch:0 step:124 [D1 loss: 0.918913, acc.: 44.53%] [D2 loss: 0.650959, acc.: 60.94%] [G loss: 3.058313]
epoch:0 step:125 [D1 loss: 1.017254, acc.: 33.59%] [D2 loss: 0.679535, acc.: 57.81%] [G loss: 2.642739]
epoch:0 step:126 [D1 loss: 0.766699, acc.: 54.69%] [D2 loss: 0.581206, acc.: 69.53%] [G loss: 3.091627]
epoch:0 step:127 [D1 loss: 0.950289, acc.: 44.53%] [D2 loss: 0.5

epoch:0 step:199 [D1 loss: 0.570509, acc.: 73.44%] [D2 loss: 0.572145, acc.: 71.88%] [G loss: 2.684153]
epoch:0 step:200 [D1 loss: 0.492736, acc.: 81.25%] [D2 loss: 0.441473, acc.: 83.59%] [G loss: 2.428051]
Instructions for updating:
Use tf.gfile.GFile.
##############
[3.10761009 2.20829648 2.61140905 3.35871226 1.6714074  6.47084663
 2.48902314 3.65173181 4.07191523 4.12570595]
##########
epoch:0 step:201 [D1 loss: 0.692909, acc.: 64.84%] [D2 loss: 0.515232, acc.: 72.66%] [G loss: 2.601077]
epoch:0 step:202 [D1 loss: 0.726727, acc.: 55.47%] [D2 loss: 0.559454, acc.: 70.31%] [G loss: 3.363036]
epoch:0 step:203 [D1 loss: 0.764815, acc.: 53.91%] [D2 loss: 0.565688, acc.: 74.22%] [G loss: 2.453548]
epoch:0 step:204 [D1 loss: 0.869215, acc.: 45.31%] [D2 loss: 0.571834, acc.: 66.41%] [G loss: 2.577363]
epoch:0 step:205 [D1 loss: 0.574358, acc.: 70.31%] [D2 loss: 0.455528, acc.: 80.47%] [G loss: 3.119158]
epoch:0 step:206 [D1 loss: 0.702540, acc.: 57.81%] [D2 loss: 0.574839, acc.: 69.53%] [

epoch:0 step:275 [D1 loss: 0.729576, acc.: 56.25%] [D2 loss: 0.522837, acc.: 75.00%] [G loss: 2.898951]
epoch:0 step:276 [D1 loss: 0.686832, acc.: 60.16%] [D2 loss: 0.496798, acc.: 78.12%] [G loss: 2.898322]
epoch:0 step:277 [D1 loss: 0.730474, acc.: 59.38%] [D2 loss: 0.466098, acc.: 81.25%] [G loss: 2.950753]
epoch:0 step:278 [D1 loss: 0.632910, acc.: 67.97%] [D2 loss: 0.524029, acc.: 71.88%] [G loss: 2.842086]
epoch:0 step:279 [D1 loss: 0.731328, acc.: 60.16%] [D2 loss: 0.462833, acc.: 82.03%] [G loss: 3.431483]
epoch:0 step:280 [D1 loss: 0.705736, acc.: 57.03%] [D2 loss: 0.531888, acc.: 76.56%] [G loss: 2.938540]
epoch:0 step:281 [D1 loss: 0.632425, acc.: 68.75%] [D2 loss: 0.383075, acc.: 87.50%] [G loss: 3.060244]
epoch:0 step:282 [D1 loss: 0.678384, acc.: 63.28%] [D2 loss: 0.494774, acc.: 78.91%] [G loss: 2.742703]
epoch:0 step:283 [D1 loss: 0.657822, acc.: 59.38%] [D2 loss: 0.489119, acc.: 74.22%] [G loss: 2.753001]
epoch:0 step:284 [D1 loss: 0.721742, acc.: 57.03%] [D2 loss: 0.3

...

epoch:29 step:13729 [D1 loss: 0.296577, acc.: 84.38%] [D2 loss: 1.287383, acc.: 50.00%] [G loss: 5.069511]
epoch:29 step:13730 [D1 loss: 0.182995, acc.: 96.09%] [D2 loss: 1.297704, acc.: 50.00%] [G loss: 4.192675]
epoch:29 step:13731 [D1 loss: 0.128243, acc.: 98.44%] [D2 loss: 1.237983, acc.: 50.00%] [G loss: 4.499652]
epoch:29 step:13732 [D1 loss: 0.210208, acc.: 96.09%] [D2 loss: 1.356505, acc.: 47.66%] [G loss: 4.676791]
epoch:29 step:13733 [D1 loss: 0.134366, acc.: 95.31%] [D2 loss: 1.303913, acc.: 49.22%] [G loss: 4.569354]
epoch:29 step:13734 [D1 loss: 0.203840, acc.: 96.09%] [D2 loss: 0.905378, acc.: 54.69%] [G loss: 4.183870]
epoch:29 step:13735 [D1 loss: 0.159566, acc.: 96.88%] [D2 loss: 1.138036, acc.: 54.69%] [G loss: 3.880578]
epoch:29 step:13736 [D1 loss: 0.110300, acc.: 99.22%] [D2 loss: 1.020618, acc.: 57.81%] [G loss: 3.677096]
epoch:29 step:13737 [D1 loss: 0.118379, acc.: 98.44%] [D2 loss: 1.194710, acc.: 47.66%] [G loss: 4.678722]
epoch:29 step:13738 [D1 loss: 0.20463

epoch:29 step:13806 [D1 loss: 0.174978, acc.: 93.75%] [D2 loss: 1.312328, acc.: 49.22%] [G loss: 4.451560]
epoch:29 step:13807 [D1 loss: 0.097127, acc.: 98.44%] [D2 loss: 1.511570, acc.: 50.78%] [G loss: 5.383948]
epoch:29 step:13808 [D1 loss: 0.151929, acc.: 96.09%] [D2 loss: 1.347843, acc.: 49.22%] [G loss: 4.573973]
epoch:29 step:13809 [D1 loss: 0.124832, acc.: 97.66%] [D2 loss: 1.160753, acc.: 48.44%] [G loss: 5.595228]
epoch:29 step:13810 [D1 loss: 0.128709, acc.: 95.31%] [D2 loss: 1.210744, acc.: 50.78%] [G loss: 6.437855]
epoch:29 step:13811 [D1 loss: 0.146033, acc.: 95.31%] [D2 loss: 1.476296, acc.: 51.56%] [G loss: 5.818913]
epoch:29 step:13812 [D1 loss: 0.114924, acc.: 96.88%] [D2 loss: 1.257362, acc.: 50.78%] [G loss: 5.166421]
epoch:29 step:13813 [D1 loss: 0.164318, acc.: 95.31%] [D2 loss: 1.361664, acc.: 50.78%] [G loss: 5.113647]
epoch:29 step:13814 [D1 loss: 0.174237, acc.: 96.88%] [D2 loss: 1.157822, acc.: 51.56%] [G loss: 4.391047]
epoch:29 step:13815 [D1 loss: 0.13038

epoch:29 step:13884 [D1 loss: 0.204657, acc.: 91.41%] [D2 loss: 1.082082, acc.: 50.00%] [G loss: 3.662339]
epoch:29 step:13885 [D1 loss: 0.133297, acc.: 96.88%] [D2 loss: 1.254666, acc.: 50.00%] [G loss: 4.267666]
epoch:29 step:13886 [D1 loss: 0.200187, acc.: 92.97%] [D2 loss: 1.177369, acc.: 53.12%] [G loss: 4.132751]
epoch:29 step:13887 [D1 loss: 0.092590, acc.: 99.22%] [D2 loss: 1.153190, acc.: 50.00%] [G loss: 4.042919]
epoch:29 step:13888 [D1 loss: 0.221612, acc.: 89.84%] [D2 loss: 1.097927, acc.: 52.34%] [G loss: 3.858974]
epoch:29 step:13889 [D1 loss: 0.122796, acc.: 94.53%] [D2 loss: 1.307335, acc.: 51.56%] [G loss: 4.914130]
epoch:29 step:13890 [D1 loss: 0.170353, acc.: 92.97%] [D2 loss: 1.310591, acc.: 50.00%] [G loss: 5.228246]
epoch:29 step:13891 [D1 loss: 0.110569, acc.: 97.66%] [D2 loss: 1.335091, acc.: 49.22%] [G loss: 4.912812]
epoch:29 step:13892 [D1 loss: 0.215310, acc.: 96.88%] [D2 loss: 1.184333, acc.: 53.12%] [G loss: 4.476894]
epoch:29 step:13893 [D1 loss: 0.17270

epoch:29 step:13962 [D1 loss: 0.141441, acc.: 97.66%] [D2 loss: 1.316405, acc.: 50.78%] [G loss: 3.620636]
epoch:29 step:13963 [D1 loss: 0.197098, acc.: 95.31%] [D2 loss: 1.142411, acc.: 53.12%] [G loss: 3.704784]
epoch:29 step:13964 [D1 loss: 0.109751, acc.: 97.66%] [D2 loss: 1.068776, acc.: 56.25%] [G loss: 4.017314]
epoch:29 step:13965 [D1 loss: 0.202392, acc.: 96.09%] [D2 loss: 1.152822, acc.: 52.34%] [G loss: 3.555218]
epoch:29 step:13966 [D1 loss: 0.360149, acc.: 88.28%] [D2 loss: 1.270821, acc.: 46.88%] [G loss: 3.676631]
epoch:29 step:13967 [D1 loss: 0.284765, acc.: 87.50%] [D2 loss: 1.159679, acc.: 52.34%] [G loss: 3.896671]
epoch:29 step:13968 [D1 loss: 0.094386, acc.: 99.22%] [D2 loss: 1.273042, acc.: 52.34%] [G loss: 5.123952]
epoch:29 step:13969 [D1 loss: 0.185981, acc.: 92.19%] [D2 loss: 1.312428, acc.: 48.44%] [G loss: 5.049335]
epoch:29 step:13970 [D1 loss: 0.212288, acc.: 91.41%] [D2 loss: 1.138790, acc.: 51.56%] [G loss: 4.588526]
epoch:29 step:13971 [D1 loss: 0.18168

epoch:29 step:14039 [D1 loss: 0.097138, acc.: 99.22%] [D2 loss: 1.257611, acc.: 48.44%] [G loss: 4.909182]
epoch:29 step:14040 [D1 loss: 0.295452, acc.: 92.97%] [D2 loss: 1.063140, acc.: 53.12%] [G loss: 4.567458]
